# NN based NGPS Localization

In [ ]:
from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/snktshrma272/LightGlue
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .

from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import cv2 as cv
from PIL import Image
import numpy

torch.set_grad_enabled(False)
images = Path("assets")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)
matcher = LightGlue(features="superpoint").eval().to(device)

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab.patches import cv2_imshow

video_path = '/content/drive/My Drive/main.mp4'
image_path = '/content/drive/My Drive/main.tif'

video = cv.VideoCapture(video_path)
if not video.isOpened():
    print(f"Error: Could not open video file at {video_path}")

image = cv.imread(image_path)
if image is None:
    print(f"Error: Could not open image file at {image_path}")
cv2_imshow(image)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import matplotlib.pyplot as plt
import numpy as np
def plot_side_by_side(img1, img2, title1="Image 0", title2="Image 1", save_path=None):

    img1_np = img1.cpu().permute(1, 2, 0).numpy()
    img2_np = img2.cpu().permute(1, 2, 0).numpy()

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img1_np)
    axs[0].set_title(title1)
    axs[0].axis('off')

    axs[1].imshow(img2_np)
    axs[1].set_title(title2)
    axs[1].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Saved plot to {save_path}")
    else:
        plt.show()
    # plt.show()

    plt.close()


In [7]:
h = 0
w = 0
tot_rot = 0
im = Image.open(image_path)
imarray = np.array(im)
imarrayB = cv.rotate(imarray, cv.ROTATE_90_CLOCKWISE)
def kernel_show(center_row, center_col, rot = 00):
    global h, w, imarray, imarrayB, tot_rot
    kernel_size = 300
    # im = Image.open(image_path)
    # imarray = np.array(im)



    h, w, c = imarray.shape

    if center_row == 0 and center_col == 0:
        center_row = h // 2
        center_col = w // 2

    margin = kernel_size // 2
    min_row, max_row = margin, h - margin
    min_col, max_col = margin, w - margin

    center_row = np.clip(center_row, min_row, max_row)
    center_col = np.clip(center_col, min_col, max_col)

    # print(center_row, center_col)
    # print(type(imarrayB.shape[1] // 2))
    tot_rot += rot
    rotMat = cv.getRotationMatrix2D((int(center_col), int(center_row)), tot_rot, 1)
    # print(rotMat)
    imarray = cv.warpAffine(imarrayB, rotMat, (imarrayB.shape[1], imarrayB.shape[0]))

    # rot3d = np.vstack((rotMat, np.array([0, 0, 1])))
    # invRot3d = np.linalg.inv(rot3d)
    # # invRot2d = invRot3d[:2, :2]
    # print("hulu: ", invRot3d)
    # corr_pt = np.matmul(invRot3d,np.array([center_col, center_row, 1]))
    # # corr_pt = invRot3d @ np.array([[center_col], [center_row], [1]])
    # print(center_col, center_row)
    # print(corr_pt)
    # imarray = imarrayB
    imarraySeg = imarray[center_row - margin : center_row + margin,
                         center_col - margin : center_col + margin,
                         :]
    imarraySeg = cv.cvtColor(imarraySeg, cv.COLOR_BGR2RGB)
    imarraySeg = cv.resize(imarraySeg, (900, 900), interpolation=cv.INTER_LINEAR)
    # print(imarray.shape)
    return imarraySeg


def map_kernel_img(x,y,base_x,base_y):
  global tot_rot
  if base_x == 0 and base_y == 0:
    base_x = h//2
    base_y = w//2

  rotMat = cv.getRotationMatrix2D((int(0), int(0)), tot_rot, 1)
  # print(rotMat)

  rot3d = np.vstack((rotMat, np.array([0, 0, 1])))
  invRot3d = np.linalg.inv(rot3d)
  # invRot2d = invRot3d[:2, :2]
  # print("hulu: ", invRot3d)
  corr_pt = np.matmul(invRot3d,np.array([y, x, 1]))
  # corr_pt = invRot3d @ np.array([[center_col], [center_row], [1]])
  # print(center_col, center_row)
  # print(corr_pt)
  print(base_x+corr_pt[1], base_y+corr_pt[0])
  return base_x+corr_pt[1], base_y+corr_pt[0]

In [ ]:
# ########## Test Code Snippet ##############
print(tot_rot)
base_x,base_y = map_kernel_img(0,200,0,0)
cv2_imshow(kernel_show(int(base_x),int(base_y),0))

In [ ]:
import torchvision.transforms as transforms
from lightglue.utils import load_image_arr
import gc
thetaDeg = 0
coord = []
cap = cv.VideoCapture(video_path)
# cap.set(cv.CAP_PROP_POS_FRAMES, 2000)
n = 0
x,y = [0,0]
base_x , base_y = [1024,712]

fps = cap.get(cv.CAP_PROP_FPS)

frames_to_skip = int(fps * (136))

cap.set(cv.CAP_PROP_POS_FRAMES, frames_to_skip)
while (cap.isOpened()):
  # if n>150:
  #   break
  try:
    base_x,base_y = map_kernel_img(y,x,base_x,base_y)
    img_kl = kernel_show(int(base_x),int(base_y),thetaDeg)
    cv2_imshow(img_kl)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    # img_kl = cv.cvtColor(kernel_show(base_x,base_y),cv.COLOR_BGR2GRAY)
    # gray_image = (image, cv2.COLOR_BGR2GRAY)

    # cv2_imshow(img_kl)
    # print(img_kl.shape)
    print("x,y,bx,by: ", x,y,base_x,base_y)
    x = 0
    y = 0
    thetaDeg /= abs(thetaDeg+0.000001) * 1
    image0 = load_image_arr(img_kl)
    feats0 = extractor.extract(image0.to(device))
    ret, frame = cap.read()
    frame = cv.resize(frame, (0,0), fx = 0.6, fy = 0.6, interpolation = cv.INTER_AREA)
    hh, ww = frame.shape[:2]
    if ret == True:
      # print(image0.ndim)
      # cv2_imshow(frame)
      if frame.ndim == 3:
        frame = frame.transpose((2,0,1))
      elif frame.ndim == 2:
        frame = frame[None]
      image1 = torch.tensor(frame / 255.0, dtype=torch.float)
      feats1 = extractor.extract(image1.to(device))
      matches01 = matcher({"image0": feats0, "image1": feats1})

      # Remove batch dimensions
      feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]

      kpts0, kpts1 = feats0["keypoints"], feats1["keypoints"]

      # Instead of matches01["matches"], use matches0 and scores
      matches0 = matches01["matches0"]              # (N,) each value is index in image1 or -1
      scores0 = matches01["matching_scores0"]       # (N,) confidence values

      # Filter good matches
      threshold = 0.5
      valid = (scores0 > threshold)

      idx0 = torch.nonzero(valid).squeeze()
      idx1 = matches0[valid]

      # Get matched keypoints
      m_kpts0 = kpts0[idx0]
      m_kpts1 = kpts1[idx1]

      print("Number of good matches", len(m_kpts0))
      if len(m_kpts0) < 20:
          continue



      M, mask = cv.findHomography(m_kpts1.cpu().numpy(), m_kpts0.cpu().numpy(), cv.RANSAC, 5.0)
      # pts = numpy.float32([[0,0],[0,frame.shape[1]],[frame.shape[0], frame.shape[1]],[frame.shape[0],0]]).reshape(-1,1,2)
      # if M is not None:
      #   rotMatHom = M[:2,:2]
      #   xScale = np.linalg.norm(rotMatHom[0])
      #   yScale = np.linalg.norm(rotMatHom[1])
      #   rotMatCorr = rotMatHom/np.array([[xScale],[yScale]])

      #   thetaRad = np.arctan2(rotMatCorr[1,0], rotMatCorr[0,0])
      #   thetaDeg = np.degrees(thetaRad)
      #   print(thetaDeg)
      # else:
      #   thetaDeg = 0

      pts = np.float32([[0, 0], [ww, 0], [ww, hh], [0, hh]]).reshape(-1, 1, 2)

      dst = cv.perspectiveTransform(pts,M)

      rect = cv.minAreaRect(numpy.int32(dst))
      thetaDeg = rect[2]
      # print(thetaDeg, "ye le bhaii")
      if thetaDeg < -45:
        thetaDeg = 90 + thetaDeg  # This normalization gives an angle between 0 and 45 degrees
      if thetaDeg > 45:
        thetaDeg = -90 + thetaDeg  # This normalization gives an angle between 0 and 45 degrees

      print(thetaDeg, "ye le bhaii")
      box = cv.boxPoints(rect)
      box = numpy.int32(box)

      # Draw the rectangle on your image
      imgF = cv.drawContours(img_kl, [box], 0, 255, 30, cv.LINE_AA)

      # print("dst: ", dst, "src: ", img_kl.shape, "m: ", M, "pts: ", pts)
      # imgF = cv.polylines(img_kl,[numpy.int32(dst)],True,255,30, cv.LINE_AA)
      x, y = numpy.mean(dst, axis=0).astype(int)[0]



      imgF = cv.circle(imgF, (x,y), 10, (0,0,255), -1)
      x,y = [x-450, y-450]
      x = x//3
      y = y//3
      transform = transforms.ToTensor()
      imgTorch = transform(imgF)

      print("new x, new y, before next loop" ,x,y)
      coord.append([base_x,base_y])



      # axes = viz2d.plot_images([imgTorch, image1])
      # # viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
      # # viz2d.add_text(0, f'Stop after {matches01["stop"]} layers', fs=20)
      # # print("done")
      # viz2d.save_plot(f'../tt{n}.png')

      plot_side_by_side(imgTorch, image1.cpu(), save_path=f'../tt{n}.png')

      print("CURRENT LOOP ENDS-------------------------------------------------------------")

      torch.cuda.empty_cache()
      # kpc0, kpc1 = viz2d.cm_prune(matches01["prune0"]), viz2d.cm_prune(matches01["prune1"])
        # viz2d.plot_images([image0, image1])
        # viz2d.plot_keypoints([kpts0, kpts1], colors=[kpc0, kpc1], ps=10)
        # plt.show()
      # print("lalala")
      n+=1
      gc.collect()

  except Exception as e:
    print(e)
    pass

In [ ]:
im = Image.open(image_path)
imarray = np.array(im)
imarray = cv.rotate(imarray, cv.ROTATE_90_COUNTERCLOCKWISE)
coord1 = np.array(coord, dtype='int32')
coordn = coord1[1:2028, [1,0]]
rcoord = coordn.reshape((-1,1,2))
mapIm = cv.polylines(imarray, [rcoord],True,255,3, cv.LINE_AA)
cv.imwrite("map.png", mapIm)

True

In [ ]:
import pickle
with open('full_rcoord.pkl','wb') as f:
  pickle.dump(rcoord, f)

In [ ]:
import cv2 as cv
img = []
for i in range(2028):
  img.append(cv.imread(f"../tt{i}" + ".png"))

h,w,d = img[1].shape
video = cv.VideoWriter('fin.mp4', cv.VideoWriter_fourcc(*'mp4v'), 30, (w,h))
for i in range(2028):
  video.write(img[i])
video.release()

In [ ]:
%cd ..

/content


In [ ]:
!rm -rf tt*